In [1]:
import pymongo
import ssl
import pandas as pd
import datetime
from bson.decimal128 import Decimal128
from bson.son import SON
import uuid
import pprint

In [2]:
# mongo_uri="mongodb://{username}@mongo-db00.dev.cld.dataeng.internal:27017/?authMechanism={auth_mechanism}&replicaSet={replica_set}&readPreference=primary&ssl={ssl}&tlsAllowInvalidCertificates=true&authSource=%24external"
#mongo_uri="mongodb://{username}@{mongo_hosts}/?authMechanism={auth_mechanism}&replicaSet={replica_set}&ssl={ssl}&tlsAllowInvalidCertificates=true&authSource=%24external"
# &readPreference=primary

mongo_uri="mongodb://{username}@{mongo_hosts}/?authMechanism=GSSAPI"
env="prod"

svx_replica_set = {
    "dev":"svx-dev1",
    "pre":"svx-preprod1",
    "prod":"svx-prod1"
}
mongo_hosts = ["mongo-db00.{env}.cld.dataeng.internal:27017".format(env=env),
               "mongo-db01.{env}.cld.dataeng.internal:27017".format(env=env),
               "mongo-db02.{env}.cld.dataeng.internal:27017".format(env=env)
              ]

svx_replica_set = svx_replica_set.get(env)
username = "s120748@AUIAG.CORP"

client=pymongo.MongoClient(
    host=mongo_hosts,
    replicaset=svx_replica_set,
    ssl=True,
    ssl_cert_reqs=ssl.CERT_NONE,
    username=username,
    authMechanism='GSSAPI',
    authSource='$external',
    readPreference="primaryPreferred",
    w="majority",
    j=True,
    retryWrites=True
    )


In [3]:
db = client.svx

In [4]:
db.list_collection_names()

['online_events',
 'customer_complaints',
 'policy_customer',
 'ds_ci_claims',
 'job_stats',
 'customer_address',
 'ds_iag_product_cover',
 'ds_svx_iag_complaints_data_harmonized',
 'ds_crods_src_sys_bus',
 'rt_svx_huon_policy',
 'customer_activity',
 'svx_iag_complaints_data_harmonized_staging',
 'consumer_invitation_responses',
 'customer_insights',
 'ds_svx_cap_policy',
 'online_activities',
 'mvdetail',
 'data_provenances',
 'vehicles_previous',
 'system.profile',
 'online_activities_staging',
 'property_structures',
 'properties',
 'ds_experian_ext_iag_pid',
 'ds_pi_claims',
 'customer_analytical_profiles',
 'ds_svx_cap_customer',
 'ds_sor_onthehouse',
 'customer',
 'ds_svx_cap_customer_staging',
 'online_events_staging',
 'products',
 'ds_svx_iag_customer_risk',
 'vehicles',
 'policy_insights',
 'ds_mdl_policy',
 'external_cache',
 'ccpi-vehicle-view']

In [5]:
collection = db.property_structures

In [ ]:
query = {
    'event': 'CREATE',
    'created_datetime': {
        '$gte': datetime.datetime(2021, 3, 21),
        '$lt': datetime.datetime(2021, 3, 22)
    }
}

cursor = collection.find(query)
result = list(cursor)
df = pd.DataFrame(result)
df

In [ ]:
df.groupby(df.created_by_user) \
  .agg({
        '_id': ['count']
    })

In [ ]:
query = {
    'provenance_id': uuid.UUID('d52b40e9-fb64-4bb4-897d-83c910c2c31c')
}

In [ ]:
cursor = collection.find(query,
                                 sort=[('created_datetime', -1)]
                                )

Property structures collection

In [ ]:
collection = db['property_structures']

In [ ]:
# number of levels
pipeline = [
    {"$group": {"_id": "$number_of_levels", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# structure type
pipeline = [
    {"$group": {"_id": "$structure_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Apartment', 'BED AND BREAKFAST', 'Commercial Farm', 'Commercial Office', 'DUPLEX', 
                'Factory Unit', 'Flat', 'GRANNY FLAT - SELF CONTAINED', 
                'HOBBY FARM OR A DOMESTIC DWELLING ON A COMMERCIAL PROPERTY', 
                'HOUSE (FREESTANDING)/TERRACE HOUSE FREE STANDING HOUSE', 'House', 'MISSING', 
                'RELOCATABLE / DEMOUNTABLE DWELLING', 'RETIREMENT VILLAGE', 'Retirement Village', 
                'SEMI-DETACHED HOUSE OR TERRACE', 'SHOPFRONT', 'STUDENT QUARTERS / NURSES QTRS', 'Semi-detached', 
                'Shed', 'Shop', 'TOWNHOUSE', 'TOWNHOUSE, VILLA OR DUPLEX', 'Terraced', 'Townhouse', 
                'UNIT/FLAT/APARTMENT (PART OF ABLOCK) UNIT OR FLAT', 'UNITS (BLOCK OF)', 'Unit', 'VILLA', 'Warehouse']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# occupancy_code
pipeline = [
    {"$group": {"_id": "$occupancy_code", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'COMBINATION OF OWNER(S) AND BOARDER(S)', 'COMBINATION OF OWNER(S) AND TENANT(S)', 
                'FOR SALE AND TENANTED', 'I own and live in the home', 
                'I rent from a landlord, by myself or with family', "It's my holiday/weekend home", 
                "It's my holiday/weekend home that I also rent out", 'No one lives in the home',
                'OWNER WHO LIVES IN THE HOME', "OWNER'S HOLIDAY HOME/WEEKENDER", 
                'RENTER - INSURING OWN CONTENTS LEASED TO TENANT', 'SEARCHING FOR TENANT UNOCCUPIED <= 30 DAYS', 
                'STRATA OWNER WHO LIVES IN THEHOME', 'STRATA TENANTS WHO LIVE IN THEHOME', 
                'TENANTS - LANDLORD INSURING CONTENTS', 'TENANTS - OWNER RENTS HOME OUTFOR HOLIDAY LETTING', 
                'UNOCCUPIED']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# primary roof type
pipeline = [
    {"$group": {"_id": "$primary_roof_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Asbestos', 'Concrete roof', 'Concrete tile', 'Fibrecement covering', 'Metal covering', 
                'Slate', 'Terracotta', 'Timber shingles']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# primary wall type
pipeline = [
    {"$group": {"_id": "$primary_wall_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Blockwork', 'Brick veneer', 'Double brick', 'Other', 'Rendered masonry', 'Sheet cladding', 
                'Weatherboard - Artificial', 'Weatherboard - Timber']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# period of construction 
pipeline = [
    {"$group": {"_id": "$period_of_construction", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Contemporary", "Federation", "PostWar", "Victorian", "War"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# quality of construction 
pipeline = [
    {"$group": {"_id": "$quality_of_construction", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Quality", "Standard"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [6]:
# bedroom count
pipeline = [
    {"$group": {"_id": "$bedroom_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15351523},
 {'_id': 3, 'count': 571915},
 {'_id': 4, 'count': 493276},
 {'_id': 5, 'count': 167279},
 {'_id': 2, 'count': 116107},
 {'_id': 6, 'count': 34572},
 {'_id': 0, 'count': 29896},
 {'_id': 1, 'count': 14183},
 {'_id': 7, 'count': 6672},
 {'_id': 8, 'count': 4824},
 {'_id': 9, 'count': 8},
 {'_id': 10, 'count': 2},
 {'_id': 54, 'count': 1},
 {'_id': 41, 'count': 1},
 {'_id': 36, 'count': 1},
 {'_id': 18, 'count': 1},
 {'_id': 12, 'count': 1},
 {'_id': 11, 'count': 1}]
FAIL: None does not exist in the allowed list of values.


In [ ]:
# bedroom small count
pipeline = [
    {"$group": {"_id": "$bedroom_small_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(1, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# bedroom medium count
pipeline = [
    {"$group": {"_id": "$bedroom_medium_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# bedroom large count
pipeline = [
    {"$group": {"_id": "$bedroom_large_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# average bedroom size
pipeline = [
    {"$group": {"_id": "$average_bedroom_size", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# construction year 
pipeline = [
    {"$group": {"_id": "$construction_year", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.property_structures.aggregate(pipeline)))

In [ ]:
# distance to bushland category 
pipeline = [
    {"$group": {"_id": "$distance_to_bushland_category", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Within 15m", "Between 16m and 100m", "More than 100m"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# distance to bushland raw value
pipeline = [
    {"$group": {"_id": "$distance_to_bushland_raw_value", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# distance to bushland 3 bands
pipeline = [
    {"$group": {"_id": "$distance_to_bushland_3bands", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Within 15m", "Between 16m and 100m", "More than 100m"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# distance to bushland 4 bands
pipeline = [
    {"$group": {"_id": "$distance_to_bushland_4bands", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Within 15m", "Between 16m and 50m", "Between 51m and 100m", "More than 100m"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# has solar panels
pipeline = [
    {"$group": {"_id": "$has_solar_panels", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, False, True]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# floor area 
pipeline = [
    {"$group": {"_id": "$floor_area", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.property_structures.aggregate(pipeline)))

In [ ]:
# verandah count 
pipeline = [
    {"$group": {"_id": "$verandah_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# lounge room count 
pipeline = [
    {"$group": {"_id": "$lounge_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 103): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# family room count 
pipeline = [
    {"$group": {"_id": "$family_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 102): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# game room count 
pipeline = [
    {"$group": {"_id": "$game_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 104): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# sun room count 
pipeline = [
    {"$group": {"_id": "$sun_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 105): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# dining room count 
pipeline = [
    {"$group": {"_id": "$dining_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# balcony count 
pipeline = [
    {"$group": {"_id": "$balcony_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# bathroom count 
pipeline = [
    {"$group": {"_id": "$bathroom_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

Properties collection

In [ ]:
collection = db['properties']

In [ ]:
# has inground pool 
pipeline = [
    {"$group": {"_id": "$has_inground_pool", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.properties.aggregate(pipeline)))

In [ ]:
# garage count
pipeline = [
    {"$group": {"_id": "$garage_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# garage types
pipeline = [
    {"$group": {"_id": "$garage_types", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Brick Garage', 'Fibro Garage', 'Metal Garage']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# carport count 
pipeline = [
    {"$group": {"_id": "$carport_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# workshop count 
pipeline = [
    {"$group": {"_id": "$workshop_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# has tennis courts 
pipeline = [
    {"$group": {"_id": "$has_tennis_court", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, False, True]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# has pool 
pipeline = [
    {"$group": {"_id": "$has_pool", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, False, True]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# has spa 
pipeline = [
    {"$group": {"_id": "$has_spa", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.properties.aggregate(pipeline)))

In [ ]:
# slope 
pipeline = [
    {"$group": {"_id": "$slope", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, "Flat/Gentle", "Severe"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# storage small count 
pipeline = [
    {"$group": {"_id": "$storage_small_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# storage large count 
pipeline = [
    {"$group": {"_id": "$storage_large_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# flat_type
pipeline = [
    {"$group": {"_id": "$flat_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'ANTENNA', 'APARTMENT', 'AUTOMATED TELLER MACHINE', 'BLOCK', 'BOATSHED', 'BUILDING', 
                'BUNGALOW', 'CAGE', 'CARPARK', 'CARSPACE', 'CLUB', 'COOLROOM', 'COTTAGE', 'DUPLEX', 'FACTORY', 'FLAT',
                'GARAGE', 'HALL', 'HOUSE', 'KIOSK', 'LOFT', 'LOT', 'MAISONETTE', 'MARINE BERTH', 'OFFICE', 
                'PENTHOUSE', 'REAR', 'RESERVE', 'ROOM', 'SECTION', 'SHED', 'SHOP', 'SHOWROOM', 'SIGN', 'SITE', 
                'STALL', 'STORE', 'STRATA UNIT', 'STUDIO', 'SUBSTATION', 'SUITE', 'TENANCY', 'TOWER', 'TOWNHOUSE',
                'UNIT', 'VAULT', 'VILLA', 'WARD', 'WAREHOUSE', 'WORKSHOP']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# level_type 
pipeline = [
    {"$group": {"_id": "$level_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'BASEMENT', 'FLOOR', 'GROUND', 'LEVEL', 'LOBBY', 'LOWER GROUND FLOOR', 'MEZZANINE', 'PARKING',
                'ROOFTOP', 'UPPER GROUND FLOOR']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# level number 
pipeline = [
    {"$group": {"_id": "$level_number", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# street suffix 
pipeline = [
    {"$group": {"_id": "$street_suffix", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'CENTRAL', 'DEVIATION', 'EAST', 'EXTENSION', 'MALL', 'NORTH', 'NORTH WEST', 'ON', 'SOUTH', 
                'SOUTH EAST', 'SOUTH WEST', 'WEST']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

In [ ]:
# dwellings count
pipeline = [
    {"$group": {"_id": "$dwellings_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")